In [339]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import csv

In [340]:
df = pd.read_csv('main_task.csv')
df.loc[df['City'] == 'Oporto', 'City'] = 'Porto'
cities = df.City.unique()

In [350]:
df = df.loc[~df.URL_TA.duplicated(keep='first')]

In [351]:
df.head(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [352]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39980 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      39980 non-null  object 
 1   City               39980 non-null  object 
 2   Cuisine Style      30701 non-null  object 
 3   Ranking            39980 non-null  float64
 4   Rating             39980 non-null  float64
 5   Price Range        26101 non-null  object 
 6   Number of Reviews  37437 non-null  float64
 7   Reviews            39980 non-null  object 
 8   URL_TA             39980 non-null  object 
 9   ID_TA              39980 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.4+ MB


In [355]:
df['URL_TA'].value_counts()

/Restaurant_Review-g186338-d8016298-Reviews-Bar_B_Que-London_England.html                            1
/Restaurant_Review-g187497-d13302247-Reviews-RomaBella-Barcelona_Catalonia.html                      1
/Restaurant_Review-g187147-d11817386-Reviews-IMAGO_Resto-Paris_Ile_de_France.html                    1
/Restaurant_Review-g187323-d7160762-Reviews-Quantuan-Berlin.html                                     1
/Restaurant_Review-g187323-d7235512-Reviews-S1_Burger_Keller-Berlin.html                             1
                                                                                                    ..
/Restaurant_Review-g187497-d11906521-Reviews-SomSacs-Barcelona_Catalonia.html                        1
/Restaurant_Review-g186338-d11985116-Reviews-Belgo_Bar_Restaurant_Kings_Cross-London_England.html    1
/Restaurant_Review-g186338-d6641339-Reviews-The_Cherry_Tree_Cafe-London_England.html                 1
/Restaurant_Review-g188113-d2495703-Reviews-Globus_Restaurant-Zurich.html

In [361]:
import time
import requests
from bs4 import BeautifulSoup
import re

URL = df['URL_TA']
HEADERS = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.152 YaBrowser/21.2.2.101 Yowser/2.5 Safari/537.36',
           'accept': '*/*'}
rest_pars = []


def get_html(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params)
    return r


def get_coord(url):
    r = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(r.text, 'html.parser')
    coord_finds = soup.find(content=re.compile(
        'https://maps.google.com/maps/api/'))
    if coord_finds:
        coord_finds = coord_finds.get('content').split(
            '&zoom')[0].split('?center=')[1].split('%2C')
    else:
        coord_finds = [[],[]]
    return coord_finds


def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    pattern = re.compile('\d+')
    meals_find = soup.find('div', string='Meals')
    adress_find = soup.find(href="#MAPVIEW")
    rub_find = soup.find('div', string=re.compile('RUB'))

    if meals_find:
        meals_find = meals_find.find_next('div').get_text().split(', ')
    else:
        meals_find = 'no_info'

    if adress_find:
        adress_find = adress_find.get_text()
    else:
        adress_find = 'no_info'

    if rub_find:
        rub_find = pattern.findall(rub_find.get_text().replace(',', ''))
        rub_max = float(rub_find[1])
        rub_min = float(rub_find[0])
    else:
        rub_find = 'no_info'
        rub_max = 'no_info'
        rub_min = 'no_info'

    rest_pars.append({'Resraurant_id': df['Restaurant_id'][df['URL_TA'] == i].item(),
                      'meals': meals_find,
                      'rest_adress': adress_find,
                      'rest_coord': get_coord('https://google.com/maps/search/'+adress_find),
                      'rub_max': rub_max,
                      'rub_min': rub_min
                      })


def parse(url):
    html = get_html(url)
    if html.status_code == 200:
        get_content(html.text)
    else:
        pass


count = 1
col = URL[145:148]

for i in col:
    parse('https://www.tripadvisor.com' + i)
    print(round(count*100/len(col), 3), '%      ',
          count, '/', len(col), end='', flush=True)
    print('\r', end='')
    count += 1

In [362]:
rest_pars

[{'Resraurant_id': 'id_8121',
  'meals': ['Late Night'],
  'rest_adress': 'Plaza Emperador Carlos V 8, 28012 Madrid Spain',
  'rest_coord': ['40.4084654', '-3.693189'],
  'rub_max': 'no_info',
  'rub_min': 'no_info'},
 {'Resraurant_id': 'id_4429',
  'meals': ['BreakfastBrunchLunchDinner Breakfast  Brunch  Lunch  Dinner '],
  'rest_adress': 'no_info',
  'rest_coord': ['70.0732779', '24.26608125'],
  'rub_max': 'no_info',
  'rub_min': 'no_info'},
 {'Resraurant_id': 'id_4110',
  'meals': ['Breakfast', 'Lunch', 'Dinner'],
  'rest_adress': 'Calle Espronceda 34, 28003 Madrid Spain',
  'rest_coord': ['40.4403981', '-3.6949632'],
  'rub_max': 'no_info',
  'rub_min': 'no_info'}]

In [363]:
data_rest = pd.DataFrame(rest_pars)
data_rest.rest_coord = data_rest.rest_coord.apply(lambda x: [float(x[0]), float(x[1])])
data_rest.to_csv('rest.csv', index = False)

In [364]:
data = pd.read_csv('rest.csv')
data

,Resraurant_id,meals,rest_adress,rest_coord,rub_max,rub_min
0,id_8121,['Late Night'],"Plaza Emperador Carlos V 8, 28012 Madrid Spain","[40.4084654, -3.693189]",no_info,no_info
1,id_4429,['BreakfastBrunchLunchDinner Breakfast Brunch...,no_info,"[70.0732779, 24.26608125]",no_info,no_info
2,id_4110,"['Breakfast', 'Lunch', 'Dinner']","Calle Espronceda 34, 28003 Madrid Spain","[40.4403981, -3.6949632]",no_info,no_info


In [365]:
print('https://www.tripadvisor.com' + URL[147])

https://www.tripadvisor.com/Restaurant_Review-g187514-d4600226-Reviews-Cafeteria_Teresa-Madrid.html
